This tutorial shows the potential of using CO2 capture resources in Macro such as Natural Gas Power CCS, SMR H2 CCS, and direct air capture (DAC). Natural Gas Power CCS and SMR H2 CCS works by specifying a CO2 capture rate (for example 0.9) of the total CO2 emitted by natural gas used in producing power and H2 which effectively reduces the total emissions by 90%. On the other hand, DAC leads to a true removal of CO2 from the atmosphere and not just a reduction of emissions. There are a total of 3 CO2 node types: 1) Captured CO2 sink node, where its input is for captured CO2 from Natural Gas Power CCS and SMR H2 CCS and atmospheric CO2 from DAC, 2) Atmospheric CO2 souce node to provide DAC with atmospheric CO2, 3) Emission CO2 sink node to account for emission by fossil sources. A system CO2 cap will now have to consider not only the CO2 emissions from the power and H2 sectors, but also the removal of atmospheric CO2 by DAC.

In [1]:
using Pkg
Pkg.activate(".")

  Activating environment at `~/Documents/GitHub/Macro/tutorials/Project.toml`


In [2]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [3]:
T = 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2_Captured=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2_Atmosphere=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [4]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(Electricity => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Hydrogen => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2_Captured => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), NaturalGas => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2_Atmosphere => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1)), PeriodLength = 8760)

In [5]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
ng_fuel_price = df[1:T,:NG_Price]/NG_MWh; # $/MWh of natural gas
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne] # MWh of hydrogen

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

ngcc_inv_cost = 65400;# $/MW
ngcc_fom_cost = 10287.0;# $/MW
ngcc_vom_cost = 3.55; #$/MW
ngcc_capsize = 250.0;
ngcc_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

ngcc_ccs_inv_cost = 66400;# $/MW
ngcc_ccs_fom_cost = 12287.0;# $/MW
ngcc_ccs_vom_cost = 3.65; #$/MW
ngcc_ccs_capsize = 250.0;
ngcc_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of electricity
ngcc_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas
ngcc_ccs_CO2_captured_rate = 0.9; #Tons of CO2 captured/Tons of CO2 produced by fuel

electric_dac_inv_cost = 9390;# $/Tonnes of CO2 captured/h
electric_dac_fom_cost = 7470;# $/Tonnes of CO2 captured/h
electric_dac_vom_cost = 22; #$/Tonnes of CO2 captured
electric_dac_power_input = 4.38; #MWh/Tonnes of CO2 captured
electric_dac_capsize = 20;# Tonnes of CO2 captured/h

electrolyzer_capsize = 2*H2_MWh # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh) # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh # $/MW of H2
electrolyzer_fom_cost = 30500/H2_MWh # $/MW of H2
electrolyzer_vom_cost = 0.0;

smr_inv_cost = 1033333/H2_MWh;# $/MW of H2
smr_fom_cost = 20500/H2_MWh;# $/MW of H2
smr_vom_cost = 0.0; #$/MW of H2
smr_capsize = 20*H2_MWh;
smr_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas

smr_ccs_inv_cost = 1133333/H2_MWh;# $/MW of H2
smr_ccs_fom_cost = 21500/H2_MWh;# $/MW of H2
smr_ccs_vom_cost = 0.0; #$/MW of H2
smr_ccs_capsize = 20*H2_MWh;
smr_ccs_heatrate = 7.43*NG_MWh; # MWh of natural gas / MWh of H2
smr_ccs_fuel_CO2 = 0.05306/NG_MWh; # Tons of CO2 / MWh of natural gas
smr_ccs_CO2_captured_rate = 0.9; #Tons of CO2 captured/Tons of CO2 produced by fuel


In [6]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
timesteps(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(timesteps(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [7]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    timesteps = timesteps(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [50000],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [8]:
solar_pv = Resource{Electricity}(;
    id = :Solar_PV,
    node = e_node,
    timesteps = timesteps(Electricity),
    subperiods = subperiods(Electricity),
    capacity_factor = solar_capacity_factor,
    can_expand = true, 
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = solar_inv_cost,
    fixed_om_cost = solar_fom_cost,
    constraints = [Macro.CapacityConstraint()]
)

Resource{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :Solar_PV, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.3834, 0.2594, 0.078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, 0.0, true, false, 85300.0, 18760.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])

We can also assigne a battery storage to the node:

In [9]:
battery = SymmetricStorage{Electricity}(;
    id = :battery,
    node = e_node,
    timesteps = timesteps(Electricity),
    subperiods = subperiods(Electricity),
    can_expand = true,
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = battery_inv_cost,
    investment_cost_storage = battery_inv_cost_storage,
    fixed_om_cost = battery_fom_cost,
    fixed_om_cost_storage = battery_fom_cost_storage,
    variable_om_cost = battery_vom_cost,
    variable_om_cost_storage = battery_vom_cost_storage,
    efficiency_injection = battery_eff_down,
    efficiency_withdrawal = battery_eff_up,
    constraints = [Macro.CapacityConstraint(),Macro.StorageCapacityConstraint()]
)

SymmetricStorage{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :battery, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 0.0, Inf, 0.0, Inf, 0.0, 0.0, true, false, 19584.0, 22494.0, 0.0, 4895.0, 5622.0, 0.0, 0.15, 0.15, 0.0, 0.92, 0.92, 0.0, 1.0, 10.0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing), Macro.StorageCapacityConstraint(missing, missing, missing)])

We also consider a hydrogen demand node:

In [10]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    timesteps = timesteps(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We model an exogenous inflow of natural gas into the system using by introducing a natural gas source node (with zero demand):

In [11]:
ng_node = SourceNode{NaturalGas}(;
    id = Symbol("NG_node"),
    timesteps = timesteps(NaturalGas),
    subperiods = subperiods(NaturalGas),
    price = ng_fuel_price,
    constraints = [Macro.DemandBalanceConstraint()]
)

SourceNode{NaturalGas}(:NG_node, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264, 18.01610783350264  …  14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564, 14.603966198369564], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We also define a CO2 emission sink node:

In [12]:
co2_node = SinkNode{CO2}(;
id = Symbol("CO2_node"),
timesteps = timesteps(CO2),
subperiods = subperiods(CO2),
constraints = [Macro.DemandBalanceConstraint()]
#### Note that if we are modeling a CO2 cap, we could add such a constraint to this sink
)


SinkNode{CO2}(:CO2_node, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Float64[], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We define an atmospheric CO2 source node to provide DAC with CO2 from the atmosphere to capture:

In [13]:
co2_atmosphere_node = SourceNode{CO2_Atmosphere}(;
    id = Symbol("CO2_Atmosphere_node"),
    timesteps = timesteps(CO2_Atmosphere),
    subperiods = subperiods(CO2_Atmosphere),
    constraints = [Macro.DemandBalanceConstraint()]
)

SourceNode{CO2_Atmosphere}(:CO2_Atmosphere_node, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Float64[], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We define a captured CO2 sink node to account for the inventory of captured CO2 (input from CO2 capture, output to SNG or potential storage):

In [14]:
co2_captured_node = SinkNode{CO2_Captured}(;
id = Symbol("CO2_Captured_node"),
timesteps = timesteps(CO2_Captured),
subperiods = subperiods(CO2_Captured),
constraints = [Macro.DemandBalanceConstraint()]
)


SinkNode{CO2_Captured}(:CO2_Captured_node, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Float64[], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)])

We are now ready to define a natural gas plant! Note that we require two stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas = CO2

This transformation has 3 edges: each edge carries a specific commodity flow

In [15]:
ngcc = Transformation{NaturalGasPower}(;
id = :NGCC,
timesteps = timesteps(Electricity),
number_of_stoichiometry_balances = 2,
constraints = [Macro.StoichiometryBalanceConstraint()]
)

push!(ngcc.TEdges,TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_capsize,
timesteps = timesteps(Electricity),
subperiods = subperiods(Electricity),
st_coeff = [ngcc_heatrate,0.0],
existing_capacity = 0.0,
investment_cost = ngcc_inv_cost,
fixed_om_cost = ngcc_fom_cost,
variable_om_cost = ngcc_vom_cost,
constraints = [CapacityConstraint()]
))

push!(ngcc.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc,
direction = :input,
has_planning_variables = false,
timesteps = timesteps(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,ngcc_fuel_CO2]
))

push!(ngcc.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc,
    direction = :output,
    has_planning_variables = false,
    timesteps = timesteps(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPower}(:NGCC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 65400.0, 10287.0, 3.55, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])
 TEdge{N

We are now ready to define a natural gas plant with CCS! Note that we require three stoichiometry balances:

Heat_Rate $\times$ Electricity = NaturalGas

Fuel_CO2 $\times$ NaturalGas $\times$ (1-CCS rate) = CO2

Fuel_CO2 $\times$ NaturalGas $\times$ CCS rate = CO2_Captured

This transformation has 4 edges: each edge carries a specific commodity flow

In [16]:
ngcc_ccs = Transformation{NaturalGasPowerCCS}(;
id = :NGCC_CCS,
timesteps = timesteps(Electricity),
number_of_stoichiometry_balances = 3,
constraints = [Macro.StoichiometryBalanceConstraint()]
)

push!(ngcc_ccs.TEdges,TEdge{Electricity}(;
id = :E,
node = e_node,
transformation = ngcc_ccs,
direction = :output,
has_planning_variables = true,
can_expand = true,
can_retire = false,
capacity_size = ngcc_ccs_capsize,
timesteps = timesteps(Electricity),
subperiods = subperiods(Electricity),
st_coeff = [ngcc_ccs_heatrate,0.0,0.0],
existing_capacity = 0.0,
investment_cost = ngcc_ccs_inv_cost,
fixed_om_cost = ngcc_ccs_fom_cost,
variable_om_cost = ngcc_ccs_vom_cost,
constraints = [CapacityConstraint()]
))

push!(ngcc_ccs.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = ngcc_ccs,
direction = :input,
has_planning_variables = false,
timesteps = timesteps(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,ngcc_ccs_fuel_CO2*(1-ngcc_ccs_CO2_captured_rate),ngcc_ccs_fuel_CO2*ngcc_ccs_CO2_captured_rate]
))

push!(ngcc_ccs.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    timesteps = timesteps(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0,0.0]
))

push!(ngcc_ccs.TEdges,TEdge{CO2_Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = ngcc_ccs,
    direction = :output,
    has_planning_variables = false,
    timesteps = timesteps(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,0.0,1.0]
))

4-element Vector{TEdge}:
 TEdge{Electricity}(:E, Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasPowerCCS}(:NGCC_CCS, 1:1:8760, 3, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 250.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0, 0.0], 0.0, Inf, 0.0, 66400.0, 12287.0, 3.65, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [17]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                timesteps = timesteps(Electricity),
                number_of_stoichiometry_balances = 1,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(electrolyzer.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    timesteps = timesteps(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [1.0],
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(electrolyzer.TEdges,TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    timesteps = timesteps(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = [electrolyzer_efficiency]
))

2-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:8760, 1, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 66.66, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0], 0.0, Inf, 0.0, 61006.090609060906, 915.0915091509152, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConst

We are now ready to define a natural gas hydrogen plant or SMR similar to a natural gas power plant

In [18]:
smr = Transformation{NaturalGasHydrogen}(;
                id = :SMR,
                timesteps = timesteps(Hydrogen),
                number_of_stoichiometry_balances = 2,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(smr.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_capsize,
    timesteps = timesteps(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [smr_heatrate,0.0],
    existing_capacity = 0.0,
    investment_cost = smr_inv_cost,
    fixed_om_cost = smr_fom_cost,
    variable_om_cost = smr_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(smr.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr,
direction = :input,
has_planning_variables = false,
timesteps = timesteps(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,smr_fuel_CO2]
))

push!(smr.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr,
    direction = :output,
    has_planning_variables = false,
    timesteps = timesteps(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogen}(:SMR, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 666.5999999999999, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0], 0.0, Inf, 0.0, 31003.090309030904, 615.0615061506151, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, An

Now, let's add a SMR hydrogen plant with CCS, similar to a natural gas power plant with CCS

In [19]:
smr_ccs = Transformation{NaturalGasHydrogenCCS}(;
                id = :SMR_CCS,
                timesteps = timesteps(Hydrogen),
                number_of_stoichiometry_balances = 3,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(smr_ccs.TEdges,TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = smr_ccs_capsize,
    timesteps = timesteps(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = [smr_ccs_heatrate,0.0,0.0],
    existing_capacity = 0.0,
    investment_cost = smr_ccs_inv_cost,
    fixed_om_cost = smr_ccs_fom_cost,
    variable_om_cost = smr_ccs_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(smr_ccs.TEdges,TEdge{NaturalGas}(;
id =  :NG,
node = ng_node,
transformation = smr_ccs,
direction = :input,
has_planning_variables = false,
timesteps = timesteps(NaturalGas),
subperiods = subperiods(NaturalGas),
st_coeff = [1.0,smr_ccs_fuel_CO2*(1-smr_ccs_CO2_captured_rate),smr_ccs_fuel_CO2*smr_ccs_CO2_captured_rate]
))

push!(smr_ccs.TEdges,TEdge{CO2}(;
    id = :CO2,
    node = co2_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    timesteps = timesteps(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,1.0,0.0]
))

push!(smr_ccs.TEdges,TEdge{CO2_Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = smr_ccs,
    direction = :output,
    has_planning_variables = false,
    timesteps = timesteps(CO2),
    subperiods = subperiods(CO2),
    st_coeff = [0.0,0.0,1.0]
))

4-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{NaturalGasHydrogenCCS}(:SMR_CCS, 1:1:8760, 3, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 666.5999999999999, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.1775180500999998, 0.0, 0.0], 0.0, Inf, 0.0, 34003.390339033904, 645.0645064506451, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), 

Let's add a direct air capture unit consuming electricity and capturing CO2 from the atmosphere into the captured CO2 inventory

In [20]:
electric_dac = Transformation{DACElectric}(;
                id = :Electric_DAC,
                timesteps = timesteps(CO2_Captured),
                number_of_stoichiometry_balances = 2,
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

push!(electric_dac.TEdges,TEdge{CO2_Captured}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = electric_dac,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electric_dac_capsize,
    timesteps = timesteps(CO2_Captured),
    subperiods = subperiods(CO2_Captured),
    st_coeff = [1.0,electric_dac_power_input],
    existing_capacity = 0.0,
    investment_cost = electric_dac_inv_cost,
    fixed_om_cost = electric_dac_fom_cost,
    variable_om_cost = electric_dac_vom_cost,
    constraints = [CapacityConstraint()]
))

push!(electric_dac.TEdges,TEdge{CO2_Atmosphere}(;
    id = :CO2_Atmosphere,
    node = co2_atmosphere_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    timesteps = timesteps(CO2_Atmosphere),
    subperiods = subperiods(CO2_Atmosphere),
    st_coeff = [1.0,0.0]
))

push!(electric_dac.TEdges,TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    timesteps = timesteps(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = [0.0,1.0]
))

3-element Vector{TEdge}:
 TEdge{CO2_Captured}(:CO2_Captured, SinkNode{CO2_Captured}(:CO2_Captured_node, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Float64[], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing)]), Transformation{DACElectric}(:Electric_DAC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 20.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 4.38], 0.0, Inf, 0.0, 9390.0, 7470.0, 22.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])
 TEdge{CO2_Atmosphere}(:CO2_Atmosphere, SourceNode{CO2_Atmosphere}(:CO2_Atmosphere_node, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Float64[], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceCons

Now that we have implemented different technologies, we can create the JuMP model.

In [21]:
components = [solar_pv; battery; ngcc; ngcc_ccs; electrolyzer; smr; smr_ccs; electric_dac];
nodes = [e_node; h2_node; ng_node; co2_node; co2_captured_node; co2_atmosphere_node];
system = [nodes;components];

In [22]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [23]:
add_planning_variables!.(components,model);

In [24]:
add_operation_variables!.(system, model);

In [25]:
add_all_model_constraints!.(system, model);

In [26]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [27]:
println(ngcc.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Electricity_E[5] - vFLOW_NaturalGas_NG[5] = 0


In [28]:
println(ngcc.constraints[1].constraint_ref[2,5])

-0.181048235160161 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] = 0


In [29]:
println(ngcc_ccs.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Electricity_E[5] - vFLOW_NaturalGas_NG[5] = 0


In [30]:
println(ngcc_ccs.constraints[1].constraint_ref[2,5])

-0.018104823516016097 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] = 0


In [31]:
println(ngcc_ccs.constraints[1].constraint_ref[3,5])

-0.16294341164414491 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_Captured_CO2_Captured[5] = 0


In [32]:
println(electrolyzer.constraints[1].constraint_ref[1,5])

vFLOW_Hydrogen_H2[5] - 0.7406666666666667 vFLOW_Electricity_E[5] = 0


In [33]:
println(smr.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Hydrogen_H2[5] - vFLOW_NaturalGas_NG[5] = 0


In [34]:
println(smr.constraints[1].constraint_ref[2,5])

-0.181048235160161 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] = 0


In [35]:
println(smr_ccs.constraints[1].constraint_ref[1,5])

2.1775180500999998 vFLOW_Hydrogen_H2[5] - vFLOW_NaturalGas_NG[5] = 0


In [36]:
println(smr_ccs.constraints[1].constraint_ref[2,5])

-0.018104823516016097 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_CO2[5] = 0


In [37]:
println(smr_ccs.constraints[1].constraint_ref[3,5])

-0.16294341164414491 vFLOW_NaturalGas_NG[5] + vFLOW_CO2_Captured_CO2_Captured[5] = 0


In [38]:
println(electric_dac.constraints[1].constraint_ref[1,5])

vFLOW_CO2_Captured_CO2_Captured[5] - vFLOW_CO2_Atmosphere_CO2_Atmosphere[5] = 0


In [39]:
println(electric_dac.constraints[1].constraint_ref[2,5])

4.38 vFLOW_CO2_Captured_CO2_Captured[5] - vFLOW_Electricity_E[5] = 0


In [40]:
using HiGHS

In [41]:
Macro.set_optimizer(model,HiGHS.Optimizer)

In [42]:
Macro.optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
100591 rows, 83080 cols, 266733 nonzeros
100591 rows, 83080 cols, 266733 nonzeros
Presolve : Reductions: rows 100591(-170980); columns 83080(-170988); elements 266733(-412047)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 17520(3.53286e+07) 0s
      57163     3.5072693726e+09 Pr: 0(0); Du: 0(7.60836e-13) 2s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 57163
Objective value     :  3.5072693726e+09
HiGHS run time      :          2.17


The installed electrolyzer capacity is:

In [43]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

0.0

The installed SMR capacity is

In [44]:
Macro.value(Macro.capacity(smr.TEdges[1]))

1039.5627

The installed SMR CCS capacity is

In [45]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

0.0

The installed solar capacity in MW is:

In [46]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed battery capacity in MW is:

In [47]:
Macro.value(Macro.capacity(battery))

100.0

The installed NGCC capacity in MW is:

In [48]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

16617.0

The installed NGCC CCS capacity in MW is:

In [49]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

0.0

The installed Electric DAC capacity in Tons CO2/h is:

In [50]:
Macro.value(Macro.capacity(electric_dac.TEdges[1]))

0.0

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [51]:
base_emissions  = Macro.value(sum(Macro.outflow(co2_node)[t] for t in timesteps(CO2)))

3.429744069856643e7

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [52]:
atm_CO2_captured  = Macro.value(sum(Macro.inflow(co2_atmosphere_node)[t] for t in timesteps(CO2)))

0.0

The net CO2 emissions (CO2 emissions - CO2 removed from the atmosphere by DAC) in tonnes are:

In [53]:
net_emissions = Macro.value(sum(Macro.outflow(co2_node)[t] for t in timesteps(CO2))) - Macro.value(sum(Macro.inflow(co2_atmosphere_node)[t] for t in timesteps(CO2)))

3.429744069856643e7

The resulting captured CO2 (DAC + Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [54]:
base_CO2_captured  = Macro.value(sum(Macro.outflow(co2_captured_node)[t] for t in timesteps(CO2_Captured)))

0.0

However, if we add a net-zero CO2 cap accounting for both positive emissions from Natural gas power and H2, and negative emissions from DAC:

In [55]:
Macro.@constraint(model,CO2Cap,sum(Macro.outflow(co2_node)[t] - Macro.inflow(co2_atmosphere_node)[t] for t in timesteps(CO2)) <= 0);

In [56]:
Macro.optimize!(model)

Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     3.5073967507e+09 Pr: 1(3.42974e+07); Du: 0(8.39553e-08) 0s
       1297     3.5073967560e+09 Pr: 1032(3.40712e+08); Du: 0(0.000149549) 6s
       2594     3.5073967560e+09 Pr: 2002(3.39131e+08); Du: 0(8.60771e-05) 12s
       3961     3.5073967560e+09 Pr: 3134(3.37138e+08); Du: 0(1.81797e-05) 18s
       5263     3.5164305682e+09 Pr: 5089(3.5438e+08); Du: 0(1.07196e-07) 24s
       6328     3.5164328404e+09 Pr: 5218(3.51497e+08); Du: 0(2.10503e-07) 30s
       7394     3.5164344243e+09 Pr: 5369(3.48103e+08); Du: 0(3.48863e-07) 35s
       8462     3.5164358128e+09 Pr: 5610(3.85044e+08); Du: 0(4.05769e-07) 40s
       9526     3.5164370930e+09 Pr: 5936(3.01761e+08); Du: 0(6.91768e-07) 46s
      10595     3.5164384127e+09 Pr: 6287(2.93505e+08); Du: 0(6.23778e-07) 51s
      11666     3.5164400540e+09 Pr: 6707(2.84358e+08); Du: 0(9.41872e-07) 57

Now, the electrolyzer capacity is:

In [57]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

0.0

The SMR capacity is:

In [58]:
Macro.value(Macro.capacity(smr.TEdges[1]))

26.330700000000093

The SMR CCS capacity is:

In [59]:
Macro.value(Macro.capacity(smr_ccs.TEdges[1]))

1013.2319999999999

Now the solar capacity is:

In [60]:
Macro.value(Macro.capacity(solar_pv))

0.0

The installed battery capacity is:

In [61]:
Macro.value(Macro.capacity(battery))

138.0

The installed NG capacity is:

In [62]:
Macro.value(Macro.capacity(ngcc.TEdges[1]))

877.9999999996264

The installed NG CCS capacity is:

In [63]:
Macro.value(Macro.capacity(ngcc_ccs.TEdges[1]))

15701.000000000373

The installed Electric DAC capacity is:

In [64]:
Macro.value(Macro.capacity(electric_dac.TEdges[1]))

1323.4018264841054

The resulting CO2 emissions from fossil fuel power and H2 production in tonnes are:

In [65]:
Macro.value(sum(Macro.outflow(co2_node)[t] for t in timesteps(CO2)))

4.15240472296013e6

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [66]:
atm_CO2_captured  = Macro.value(sum(Macro.inflow(co2_atmosphere_node)[t] for t in timesteps(CO2)))

4.1524047229603715e6

The net CO2 emissions (CO2 emissions - CO2 removed from the atmosphere by DAC) in tonnes are:

In [67]:
net_emissions = Macro.value(sum(Macro.outflow(co2_node)[t] for t in timesteps(CO2))) - Macro.value(sum(Macro.inflow(co2_atmosphere_node)[t] for t in timesteps(CO2)))

-2.416782081127167e-7

The resulting captured CO2 (DAC + Power CCS + H2 CCS) in tonnes that can be transported to storage are:

In [68]:
Macro.value(sum(Macro.outflow(co2_captured_node)[t] for t in timesteps(CO2_Captured)))

4.146761719728147e7

We see that in a net-zero emission case with negative emissions from DAC, a combination of Natural Gas Power CCS and SMR H2 CCS with DAC is deployed.